<a href="https://colab.research.google.com/github/Ansave/M806_XR_AI/blob/main/CUDA_Homework/CW/CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Постановка задачи

1. Обучить модель на трёх независимых процессах (обучения моделей производятся на непересекающихся множествах обучающих данных)

2. Для набора тестовых данных провести ансамблевое голосование



Датасет: *Mnist*

Модель: *CNN*


In [256]:
# ! pip install mpi4py

# Подключение зависимостей

In [257]:
%%writefile CW.py

import torch
import numpy as np
import operator
import mpi4py

from mpi4py import MPI
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.utils.data as data_utils

Overwriting CW.py


# Загрузка датасета MNIST

In [258]:
%%writefile -a CW.py

train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)

test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

loaders = {}

loaders['train'] = torch.utils.data.DataLoader(
  train_data, 
  batch_size=100, 
  shuffle=True, 
  num_workers=1
)

loaders['test'] = torch.utils.data.DataLoader(
  test_data, 
  batch_size=100, 
  shuffle=True, 
  num_workers=1
)


Appending to CW.py


# Определение CNN модели

In [259]:
%%writefile -a CW.py

import torch.nn as nn

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.out = nn.Linear(32 * 7 * 7, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)       
        
        output = self.out(x)
        return output, x    

Appending to CW.py


Обучение модели

In [260]:
%%writefile -a CW.py

from torch.autograd import Variable

def train(epochs_count, cnn, loaders_trains):
  loss_func = nn.CrossEntropyLoss()
  optimizer = optim.Adam(cnn.parameters(), lr = 0.01)
  
  cnn.train()
  total_step = len(loaders_trains)
  
  for epoch in range(epochs_count):
    for i, (images, labels) in enumerate(loaders_trains): 
      batch_x = Variable(images)
      batch_y = Variable(labels)
      output = cnn(batch_x)[0]
      
      loss = loss_func(output, batch_y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

Appending to CW.py


Оценка точности обученной модели

In [261]:
%%writefile -a CW.py
def evaluate(cnn_for_test):
  cnn_for_test.eval()
  
  with torch.no_grad():
    for images, labels in loaders['test']:
      test_output, last_layer = cnn_for_test(images)
      predicted = torch.max(test_output, 1)[1].data.squeeze()
    
    return (predicted == labels).sum().item() / float(labels.size(0))

Appending to CW.py


# Параллельная программа


In [262]:
%%writefile -a CW.py

comm = MPI.COMM_WORLD 
numprocs = comm.Get_size()
rank = comm.Get_rank() 

Appending to CW.py


## Установка датасета

In [263]:
%%writefile -a CW.py

train_part = int(len(train_data) / numprocs)
image_indices = torch.arange(6000)

cut_train = data_utils.Subset(train_data, image_indices)
trains = random_split(cut_train, [int(6000/(numprocs-1)) for i in range(numprocs-1)])

Appending to CW.py


Управляющие параметры

In [264]:
%%writefile -a CW.py

numbers_count = 20
num_epochs = 5

Appending to CW.py


## Исполняющие процессы

In [265]:
%%writefile -a CW.py

if rank != 0:

Appending to CW.py


Обучение

In [266]:
%%writefile -a CW.py
  cnn = CNN()
  subset = DataLoader(
    trains[rank-1],                              
    batch_size = 100,                              
    shuffle = True,                              
    num_workers = 1)

  train(num_epochs, cnn, subset)
  print(f'Точность модели {rank} процесса: {evaluate(cnn)}')


Appending to CW.py


Предсказание

In [267]:
%%writefile -a CW.py
  
  images = comm.recv(source = 0) 
  test_output, last_layer = cnn(images[:numbers_count])
  
  predicted = torch.max(test_output, 1)[1].data.numpy().squeeze()
  preds_proc = comm.send(predicted, dest = 0)


Appending to CW.py


## Управляющий процесс

In [268]:
%%writefile -a CW.py

else:

Appending to CW.py


Передача экземпляра тестовых данных на исполняющие процессы

In [269]:
%%writefile -a CW.py

  sample = next(iter(DataLoader(
    test_data,                              
    batch_size = 100,                              
    shuffle = True,                              
    num_workers = 1)))

  images, labels = sample
  actual_number = labels[:numbers_count].numpy()

  for k in range(1, numprocs):
    comm.send(images, dest = k)

Appending to CW.py


Агрегация полученных результатов

In [270]:
%%writefile -a CW.py

  predictions = []
  for k in range(1, numprocs):
    predicted = comm.recv(source = k)
    predictions.append(predicted)

  for k in range(1, numprocs):
    print(f'Предсказания {k} процесса: {predictions[k - 1]}')

Appending to CW.py


Ансамблевое голосование

In [271]:
%%writefile -a CW.py

  predicted_result = []  
  for num in range(len(predictions[0])):
    check_dict = {}
    for k in range(1, numprocs):
      current_number = predictions[k - 1][num] 
      num_count = check_dict.get(current_number, 0)
      num_count = num_count + 1
      check_dict[current_number] = num_count

    best_num = max(check_dict.items(), key = operator.itemgetter(1))[0]
    predicted_result.append(best_num)

  print(f'\nРезультат голосования:\t {np.array(predicted_result)}')
  print(f'Реальные значения:\t {actual_number}')

Appending to CW.py


In [272]:
%%writefile -a CW.py

MPI.Finalize()

Appending to CW.py


# Вызов параллельной программы

In [273]:
!mpirun -np 4 --allow-run-as-root python CW.py

Точность модели 3 процесса: 0.98
Точность модели 1 процесса: 0.97
Точность модели 2 процесса: 0.97
Предсказания 1 процесса: [0 9 7 2 1 3 8 3 9 4 2 7 2 7 0 8 7 7 4 5]
Предсказания 2 процесса: [0 9 7 8 1 0 8 3 9 4 2 7 2 7 0 8 7 7 4 5]
Предсказания 3 процесса: [0 9 7 2 1 0 8 3 9 4 2 7 2 7 0 8 7 7 4 5]

Результат голосования:	 [0 9 7 2 1 0 8 3 9 4 2 7 2 7 0 8 7 7 4 5]
Реальные значения:	 [0 9 7 7 1 3 8 3 9 4 2 7 2 7 0 8 7 7 4 5]
